In [5]:
%pylab inline
from PIL import Image
import glob
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np


Populating the interactive namespace from numpy and matplotlib


In [6]:
images_path = glob.glob("./data/rit/Harsh2/*.jpg")
images_path.extend(glob.glob("./data/rit/Harsh2/*.JPG"))
images_path.extend(glob.glob("./data/rit/Harsh2/*.png"))

In [7]:
def images_to_sprite(data):
    """
    Creates the sprite image along with any necessary padding
    Source : https://github.com/tensorflow/tensorflow/issues/6322
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [17]:
def populate_img_arr(size=(100,100),should_preprocess= False):
    arr = []
    for i,img_path in enumerate(images_path):
        img = image.load_img(img_path, target_size=size)
        x = image.img_to_array(img)
        arr.append(x)
    arr = np.array(arr)
    if should_preprocess:
        arr = preprocess_input(arr)
    return arr        

In [18]:
model = InceptionV3(include_top=True)
model.layers.pop()
model.outputs = [model.layers[-1].output]
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_95[0][0]     
___________________________________________________________________________________________

In [19]:
img_arr = populate_img_arr(size=(299,299),should_preprocess=True)

In [20]:
preds = model.predict(img_arr,batch_size=64)

In [21]:
preds.tofile("./oss_data/tensor.bytes")

In [23]:
raw_imgs = populate_img_arr(size=(100,100),should_preprocess=False)

In [24]:
plt.imsave('./oss_data/sprites.png',images_to_sprite(raw_imgs))